### Описание проекта
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

In [1]:
import pandas as pd

data = pd.read_csv('data.csv')

data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
data.info()
print()
print('-----------------------------')
print()
data.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB

-----------------------------



(21525, 12)

## Вывод

- Данные содержатся в 12 столбцах, которые представляют собой категории заемщиков;
- Всего 21525 данных, следовательно данная таблица содержит информацию о 21525 заемщиков;
- Имеется наличие пропусков в данных;
- Таблица содержит различные типы данных (float64, int64, object)
- Имеется повторяющаяся информация, которую можно извлечь из данных (например: **family_status** и **family_status_id**)	
- Колонка **total_income** (ежемесячный доход) содержит слишком подробную информацию, такая точность необязательна;
- Колонка **dob_years** (возраст) содержит цифру, а не дату рождения, что требует проверки достоверности данных (необходима "свежая" информация на сегодняшний день);
- **days_employed**(общий трудовой стаж в днях) содержит как отрицательные, так и положительные значения;
- **education** имеет разные форматы записи данных.


# Этап 2 

## Предобработка данных

Известно что данные содержат пропуски, но просто так мы не можем удалить значения NaN или заменить их нулевыми значениями, так как мы не добьёмся точности нашего анализа. Поэтому необходимо изучить пропуски подробнее.

In [3]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски содержат колоки **"days_employed"** и **"total_income"**.
Убедимся что данные в этих столбцах не содержат нулей.

In [4]:
data[data["days_employed"] == 0].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [5]:
data[data["total_income"] == 0].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

В данных столбцах нет нулевых значений, поэтому можно заменять NaN, но заменять пропуски на **0** будет неразумно. Для начала заменим значения в столбце **total_income** медианными значениями этого столбца. Столбец **days_employed** пока не будем трогать, так как сначала нужно разобраться с отрицательными значениями в его данных.

Мы имеем дело с количественными переменными. Пропуски в таких переменных заполняют характерными значениями. Это значения, характеризующие состояние выборки, — набора данных, выбранных для проведения исследования. Чтобы примерно оценить типичные значения выборки, годятся среднее арифметическое или медиана. Среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства. Когда в выборке присутствуют выдающиеся значения, лучше использовать медиану.

In [6]:
median_t_income = data['total_income'].median()
median_t_income

145017.93753253992

Данные в столбце **days_employed** заменим на нулевые значения, так как мы убедились что их нет в исходной таблице ранее. Позже вернёмся к этому и с помощью метода **loc()** заменим их на значения медианы столбца.



In [7]:

data['days_employed'].fillna(value=0, inplace=True)
data['total_income'].fillna(value=median_t_income, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Таким образом в первой фазе предобработки данных мы избавились от пропусков, но позже ещё вернёмся к этой задаче. Убедимся в этом ещё раз.

In [8]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [9]:
data_zero = data[ (data['days_employed'] == 0) & (data['total_income'] == median_t_income) ]
data_zero.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,0.0,65,среднее,1,гражданский брак,1,M,пенсионер,0,145017.937533,сыграть свадьбу
26,0,0.0,41,среднее,1,женат / замужем,0,M,госслужащий,0,145017.937533,образование
29,0,0.0,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,145017.937533,строительство жилой недвижимости
41,0,0.0,50,среднее,1,женат / замужем,0,F,госслужащий,0,145017.937533,сделка с подержанным автомобилем
55,0,0.0,54,среднее,1,гражданский брак,1,F,пенсионер,1,145017.937533,сыграть свадьбу


In [10]:
data_zero.shape

(2174, 12)

Получается пропущенниые значения в двух столбцах совпадали друг с другом, следовательно их появление в данных не было случайным, хотя взаимосвязи с другими критериями у них нет. Пустые значения соответсвуют потенциальному заёмщику либо не имеющего дохода, либо без стажа работы, что имеет взаимосвязь. 

In [11]:
n1 = data_zero['days_employed'].count()
n2 = data['days_employed'].count()

print(f'Данные таблицы с пропущенными значениями составляет {n1/n2:.2%}')

Данные таблицы с пропущенными значениями составляет 10.10%


## Вывод

Пропуски присутствуют в колонках **'days_employed'** и **'total_income'** - вероятно, кто-то никогда не работал, не имел подтвержденного дохода, либо предпочёл пропустить этот пункт при анкетировании. Явной зависимости от других вводных не наблюдается.




# Этап 3

## Замена типа данных

Вещественный тип данных **(float)** в колонке **days_employed** непоказателен. Заменим его на целочисленный **(int)**, уберем минусы. Есть подозрение, что часть данных предоставлена не в днях, а в часах, так как эти значения очень большие и могут составлять сотни лет.

Изучим подробнее столбец с данными **days_employed** и избавимся от отрицательных значений методом **abs**

In [12]:
def f_days(day):
    abs_day = abs(day)
    return(int(abs_day))

def f_years(day):
    years = day/365
    return(int(years))

data['days_employed_new'] = data['days_employed'].apply(f_days)
data['years_employed'] =  data['days_employed_new'].apply(f_years)

data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_new,years_employed
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,8437,23
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,4024,11
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,5623,15
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,4124,11
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,340266,932
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,926,2
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,2879,7
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,152,0
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,6929,18
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,2188,5


Мы избавились от минусов в значениях стажа заёмщиков, и также проверили адекватность этих данных. Выходит некоторые заёмщики работают больше сотни лет, что является сомнительным. Видимо при составлении этих данных была допущена ошибка и вместо дней были указаны часы. Это наиболее из реальных версий, поэтому это стоит проверить.

In [13]:
data[(data['years_employed'] >= data['dob_years'])].head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_new,years_employed
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,340266,932
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля,400281,1096
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью,338551,927
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости,363548,996
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью,335581,919
35,0,394021.072184,68,среднее,1,гражданский брак,1,M,пенсионер,0,77805.677436,на проведение свадьбы,394021,1079
50,0,353731.432338,63,среднее,1,женат / замужем,0,F,пенсионер,0,92342.730612,автомобили,353731,969
56,0,370145.087237,64,среднее,1,вдовец / вдова,2,F,пенсионер,0,149141.043533,образование,370145,1014
71,0,338113.529892,62,среднее,1,женат / замужем,0,F,пенсионер,0,43929.696397,автомобили,338113,926
78,0,359722.945074,61,высшее,0,женат / замужем,0,M,пенсионер,0,175127.646000,сделка с автомобилем,359722,985


In [14]:
data[(data['years_employed'] >= data['dob_years'])]['income_type'].count()

3529

In [15]:
data[(data['income_type'] == 'пенсионер')]['gender'].count()

3856

In [16]:
data[(data['years_employed'] >= data['dob_years']) & (data['income_type'] == 'пенсионер')]['gender'].count()

3446

Получаем очень интересную информацию. Что преимущественно ошибка в стаже есть у пенсионеров, но не все они входят в этот список, так как их в данных больше 3856 > 3446. Остальные в этом списке люди работают дольше по времени, чем их возраст, чего быть просто не может. Если учитывать минимальный возраст с которого можно работать 14 лет, то определим количесво людей, чей стаж больше их возраста без 14 лет.

In [17]:
data[(data['years_employed'] >= (data['dob_years'] - 14))]['income_type'].count()

3545

теперь точно всё, мы определили аномальные данные в таблице и их 3545.

Переведём значение стажа этих строк из часов в дни, разделив на 24.

In [24]:
def f_days_new (data):
    
    for i in range(len(data)):
        if (data['years_employed'][i] > (int(data['dob_years'][i]) - 14)):
            data['years_employed'][i] = int(data['years_employed'][i]/24)
            data['days_employed_new'][i] = int(data['days_employed_new'][i]/24)
    return data

f_days_new(data)


data.head(10)




C:\Users\ruk19\AppData\Local\Temp/ipykernel_460/273663883.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['years_employed'][i] = int(data['years_employed'][i]/24)
C:\Users\ruk19\AppData\Local\Temp/ipykernel_460/273663883.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['days_employed_new'][i] = int(data['days_employed_new'][i]/24)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_new,years_employed
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,8437,23
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,4024,11
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,5623,15
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,4124,11
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,14177,38
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,926,2
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,2879,7
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,152,0
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,6929,18
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,2188,5


Сделаем проверку:

In [25]:
data[(data['years_employed'] > (data['dob_years'] - 14))]['income_type'].count()

101

In [26]:
data[(data['years_employed'] > (data['dob_years'] - 14))].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_new,years_employed
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль,601,1
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем,4,0
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью,3,0
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости,690,1
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль,2,0
1149,0,-934.654854,0,среднее,1,женат / замужем,0,F,компаньон,0,201852.430096,покупка недвижимости,1,0
1175,0,370879.508002,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования,643,1
1386,0,-5043.219890,0,высшее,0,женат / замужем,0,M,госслужащий,0,240523.618071,сделка с автомобилем,8,0
1890,0,0.000000,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,145017.937533,жилье,0,0
1898,0,370144.537021,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля,642,1


Интересный вывод мы получаем. Избавляясь от ошибки, мы обнаружили другую ошибки в исходных данных, а конкретнее это отсутствие возраста клиентов, вместо этого стоят нули, тогда учтём это и оставим их стаж без изменения.

Преобразуем **total_income** в целочисленный тип **(int)**

In [27]:
data['total_income'] = data['total_income'].astype(int)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_new,years_employed
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,8437,23
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,4024,11
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,5623,15
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,4124,11
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,14177,38
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,926,2
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,2879,7
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование,152,0
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,6929,18
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,2188,5


Колонка **education** требует приведения к единому формату (нижнему регистру):

In [28]:
data['education'] = data['education'].apply(str.lower)
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Проверим столбец children:

In [29]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Как оказалось, есть 47 строк, у которых (-1) ребенок. Выведем на экран эту выборку:

In [30]:
data[data['children'] == (-1)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_new,years_employed
291,-1,-4417.703588,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816,профильное образование,4417,12
705,-1,-902.084528,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882,приобретение автомобиля,902,2
742,-1,-3174.456205,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268,дополнительное образование,3174,8
800,-1,349987.852217,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293,дополнительное образование,14582,39
941,-1,0.000000,57,среднее,1,женат / замужем,0,F,пенсионер,0,145017,на покупку своего автомобиля,0,0
1363,-1,-1195.264956,55,среднее,1,женат / замужем,0,F,компаньон,0,69550,профильное образование,1195,3
1929,-1,-1461.303336,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121,покупка жилья,1461,4
2073,-1,-2539.761232,42,среднее,1,в разводе,3,F,компаньон,0,162638,покупка жилья,2539,6
3814,-1,-3045.290443,26,среднее,1,гражданский брак,1,F,госслужащий,0,131892,на проведение свадьбы,3045,8
4201,-1,-901.101738,41,среднее,1,женат / замужем,0,F,госслужащий,0,226375,операции со своей недвижимостью,901,2


Скорее всего данная ошибка была допущена из-за неправильного формата при выгрузке данных. Исправим это:

In [31]:
def children(children):
    return abs(children)

data['children'] = data['children'].apply(children)

data['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

Также бросается в глаза значение 20, почему именно 20 детей, когда ни у кого нет больше 5 и меньше 20 детей в семье? И раз 20 детей имеет ни одна семья, тут явно допущена ошибка

In [32]:
data[data['children'] == (20)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_new,years_employed
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334,покупка жилья,880,2
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998,покупка недвижимости,855,2
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518,получение образования,3310,9
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474,операции с коммерческой недвижимостью,2714,7
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739,на покупку автомобиля,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,-1240.257910,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524,свой автомобиль,1240,3
21325,20,-601.174883,37,среднее,1,женат / замужем,0,F,компаньон,0,102986,профильное образование,601,1
21390,20,0.000000,53,среднее,1,женат / замужем,0,M,компаньон,0,145017,покупка жилой недвижимости,0,0
21404,20,-494.788448,52,среднее,1,женат / замужем,0,M,компаньон,0,156629,операции со своей недвижимостью,494,1


Вывод на основе этого можно сделать вывод, что здесь допустили ошибку при форматировании данных и на самом деле детей у этих клиентов должно быть 2, тогда:

In [43]:
# def chil_20 (data):
#    for i in range(len(data)):
#        if data['children'][i] == 20:
#           data['children'][i] = 2
#  return(data)
#chil_20 (data)
#data['children'].value_counts()

def chil_20(children):
    if children == 20:
        return 2
    return children
data['children'] = data['children'].apply(chil_20(children))

data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Так количество детей выглядит более правдоподобно. Хотя мы уже нашли ошибку с возрастом клиентов равным 0, ещё раз проверим этот столбец:

In [44]:
data['dob_years'].value_counts().sort_index()

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

Так как нулевых значений здесь 101, это не такая большая часть всех данных (21525) - 0,4% 

Можно просто опустить эти строки при дальнейшей работы с таблицей.

In [45]:
data = data.loc[data['dob_years'] > 0]
print(data['dob_years'].count())
print(data[(data['dob_years'] ==0)]['children'].count())

21424
0


## Вывод

Теперь данные приведены к единому формату, за исключением **days_employed** - они переведены в годы и выведены в отдельный столбец **years_employed**.





# 4 этап

## Обработка дубликатов

In [46]:
data.duplicated().sum()

71

In [47]:
data[data.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_new,years_employed
2849,0,0.0,41,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для семьи,0,0
3290,0,0.0,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу,0,0
4182,1,0.0,34,высшее,0,гражданский брак,1,F,сотрудник,0,145017,свадьба,0,0
4851,0,0.0,60,среднее,1,гражданский брак,1,F,пенсионер,0,145017,свадьба,0,0
5557,0,0.0,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,0.0,64,среднее,1,женат / замужем,0,F,пенсионер,0,145017,дополнительное образование,0,0
21032,0,0.0,60,среднее,1,женат / замужем,0,F,пенсионер,0,145017,заняться образованием,0,0
21132,0,0.0,47,среднее,1,женат / замужем,0,F,сотрудник,0,145017,ремонт жилью,0,0
21281,1,0.0,30,высшее,0,женат / замужем,0,F,сотрудник,0,145017,покупка коммерческой недвижимости,0,0


In [48]:
data.drop_duplicates(inplace=True)

data.duplicated().sum()

0

In [49]:
data[data.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_new,years_employed


In [50]:
data.shape

(21353, 14)

## Вывод

Есть некоторые дубликаты, от которых мы избавились.

# 5 этап

## Лемматизация

Найдём все уникальные значения в колонке purpose:

In [51]:
purpose_unprocessed = data['purpose'].unique()
purpose_unprocessed

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Лемматизируем:

In [52]:
! pip install pymystem3

In [53]:
# Импортируем pymystem3
from pymystem3 import Mystem
m = Mystem()

# Создаем дополнительный словарь чтобы записать в него неочищенные списки после лемматизации:
purpose_processed = []
for i in purpose_unprocessed:
    i = purpose_processed.append(m.lemmatize(i))

# Избавимся от лишних символов для увеличения читаемости списка:  
for row in purpose_processed:
    for element in row:
        if element == ' ':
            row.remove(' ')
        elif element == '\n':
            row.remove('\n')
        else:
            pass

# ... и выведем обработанный список без лишних символов:
purpose_processed

[['покупка', 'жилье'],
 ['приобретение', 'автомобиль'],
 ['дополнительный', 'образование'],
 ['сыграть', 'свадьба'],
 ['операция', 'с', 'жилье'],
 ['образование'],
 ['на', 'проведение', 'свадьба'],
 ['покупка', 'жилье', 'для', 'семья'],
 ['покупка', 'недвижимость'],
 ['покупка', 'коммерческий', 'недвижимость'],
 ['покупка', 'жилой', 'недвижимость'],
 ['строительство', 'собственный', 'недвижимость'],
 ['недвижимость'],
 ['строительство', 'недвижимость'],
 ['на', 'покупка', 'подержать', 'автомобиль'],
 ['на', 'покупка', 'свой', 'автомобиль'],
 ['операция', 'с', 'коммерческий', 'недвижимость'],
 ['строительство', 'жилой', 'недвижимость'],
 ['жилье'],
 ['операция', 'со', 'свой', 'недвижимость'],
 ['автомобиль'],
 ['заниматься', 'образование'],
 ['сделка', 'с', 'подержанный', 'автомобиль'],
 ['получение', 'образование'],
 ['автомобиль'],
 ['свадьба'],
 ['получение', 'дополнительный', 'образование'],
 ['покупка', 'свой', 'жилье'],
 ['операция', 'с', 'недвижимость'],
 ['получение', 'высокий',

In [54]:

purpose_dict = []
for row in purpose_processed:
    purpose_dict.append(row[-1])
purpose_dict

['жилье',
 'автомобиль',
 'образование',
 'свадьба',
 'жилье',
 'образование',
 'свадьба',
 'семья',
 'недвижимость',
 'недвижимость',
 'недвижимость',
 'недвижимость',
 'недвижимость',
 'недвижимость',
 'автомобиль',
 'автомобиль',
 'недвижимость',
 'недвижимость',
 'жилье',
 'недвижимость',
 'автомобиль',
 'образование',
 'автомобиль',
 'образование',
 'автомобиль',
 'свадьба',
 'образование',
 'жилье',
 'недвижимость',
 'образование',
 'автомобиль',
 'автомобиль',
 'образование',
 'образование',
 'сдача',
 'автомобиль',
 'жилье',
 'образование']

In [55]:
set(purpose_dict)

{'автомобиль',
 'жилье',
 'недвижимость',
 'образование',
 'свадьба',
 'сдача',
 'семья'}

Слова "жилье", "семья" и "сдача" исходя из словаря purpose_dict относятся к "недвижимость". Теперь применим лемматизацию к столбцу purpose в data:

# 6 Этап

## Категоризация данных

Один из вопросов, на которые требуется дать ответ - это влияет ли уровень дохода на потенциальное наличие задолженности. Нужно создать категории для дохода. Посмотрим какие ненулевые значения есть в этой колонке:

In [58]:
employed_only = data[ (data['days_employed'] != 0) & (data['total_income'] != 0) ]
employed_only

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_new,years_employed
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,8437,23
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,4024,11
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,5623,15
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,4124,11
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,14177,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,4529,12
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,14330,39
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,2113,5
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,3112,8


In [65]:
employed_only['total_income'].sort_values(ascending=False)

12412    2265604
19606    2200852
9169     1726276
20809    1715018
17178    1711309
          ...   
14276      21895
1598       21695
16174      21367
13006      21205
14585      20667
Name: total_income, Length: 19260, dtype: int32

In [66]:
employed_only['total_income'].mean()

167464.72367601245

In [67]:
employed_only['total_income'].median()

145011.0

Доход заемщиков распределен более-менее равномерно, так как средние значения и медиана отличаются не в разы. 

Распределю клиентов по разным категориям:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

In [69]:
def category(total_income):
    if total_income < 30000:
            return 'Е'
    if total_income <= 50000:
            return 'D'
    if total_income <= 200000:
            return 'C'
    if total_income <= 1000000:
            return 'B'    
    return 'A'

# Проверим её на нескольких значениях:
print(category(5000))
print(category(50000))
print(category(500000))
print(category(5000000))

Е
D
B
A


Применим функцию к датафрейму employed_only - там, где нет нулевых значений в пункте о доходах.

In [71]:

employed_only['income_category'] = employed_only['total_income'].apply(category)
employed_only.head(10)

C:\Users\ruk19\AppData\Local\Temp/ipykernel_460/3271694050.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  employed_only['income_category'] = employed_only['total_income'].apply(category)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_new,years_employed,income_category
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,8437,23,B
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,4024,11,C
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,5623,15,C
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,4124,11,B
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,14177,38,C
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,926,2,B
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,2879,7,B
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,152,0,C
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,6929,18,C
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,2188,5,C


In [72]:
employed_only.groupby('income_category').mean()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income,days_employed_new,years_employed
income_category,,,,,,,,,
A,0.720000,-3358.516917,45.280000,0.280000,0.920000,0.080000,1.339118e+06,3358.040000,8.680000
B,0.502094,37839.956459,42.554138,0.692722,0.950947,0.070788,2.866826e+05,3393.109472,8.790429
C,0.472058,69255.888211,43.613285,0.861805,0.985126,0.085415,1.255365e+05,4238.080578,11.095307
D,0.445402,171552.118288,49.537356,1.000000,0.721264,0.060345,4.290583e+04,6992.718391,18.652299
Е,0.272727,247039.031639,55.000000,1.090909,0.636364,0.090909,2.579482e+04,9640.909091,25.909091


## Вывод

Мы применили категоризацию к данным в колонке total_income.Доход - это действительно показатель, который можно разделить по статистике для страны или другим экономическим показателям. 

# 7 Этап 

Определим: **Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Для большей наглядности уберу из таблицы лишнюю на данный момент информацию

In [82]:
data_new = employed_only[['children','dob_years','total_income','days_employed_new','income_category','debt']]
data_new.head(10)

,children,dob_years,total_income,days_employed_new,income_category,debt
0,1,42,253875,8437,B,0
1,1,36,112080,4024,C,0
2,0,33,145885,5623,C,0
3,3,32,267628,4124,B,0
4,0,53,158616,14177,C,0
5,0,27,255763,926,B,0
6,0,43,240525,2879,B,0
7,0,50,135823,152,C,0
8,2,35,95856,6929,C,0
9,0,41,144425,2188,C,0


Сделаем таблицу в которой можно будет посмотреть долю по невозвращаемости среди заемщиков с детьми:

По умолчанию функция **concat()** работает на **axis=0** и возвращает объект Series. Если задать 1 значением **axis**, то результатом будет объект Dataframe.

In [92]:
def uni_func (name_column):
    
    data_func = employed_only[[name_column,'debt']]
    n1 = data_func.groupby(name_column).sum()
    n2 = data_func.groupby(name_column).count()
    n3 = (n1/n1.sum())*100
    n4 =n1/n2 *100
    data_final = pd.concat([n1, n2, n3, n4], axis =1)
    data_final.columns =['debt', 'total', 'share', 'percentage']
    return data_final
    

uni_func ('children')

,debt,total,share,percentage
children,,,,
0,947,12649,60.588612,7.486758
1,406,4372,25.975688,9.286368
2,185,1904,11.836212,9.716387
3,22,293,1.407550,7.508532
4,3,34,0.191939,8.823529
5,0,8,0.000000,0.000000


## Вывод

Исходя из таблицы, с ростом количества детей процент задолженности падает, то есть чем больше детей, тем меньше вероятность образования задолженности. При этом процент просрочек у заемщиков с одним и двумя детьми являются самыми рискованными группами - 9.3% и 9.7% (от общего числа займов) людей соответственно в этой группе имеют просрочку по платежам, а самыми благонадежными являются люди без детей или с тремя детьми, имея 7.5% просрочек (от общего числа займов) каждый.

**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [97]:
uni_func('family_status').sort_values(by ='debt', ascending=False)

,debt,total,share,percentage
family_status,,,,
женат / замужем,842,11098,53.870761,7.586953
гражданский брак,337,3717,21.561100,9.066451
Не женат / не замужем,253,2510,16.186820,10.079681
в разводе,76,1074,4.862444,7.076350
вдовец / вдова,55,861,3.518874,6.387921


## Вывод

Разведенные и вдовы(-цы) - наиболее склонны возвращать кредиты вовремя, тогда как женатые/замужние люди - самая рискованная группа после тех, кто состоит в гражданском браке или не состоит в браке вовсе. Самая "рискованная" группа для выдачи займа - свободные и состоящие в гражданском браке люди.

**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [99]:
uni_func('income_category')

,debt,total,share,percentage
income_category,,,,
A,2,25,0.127959,8.000000
B,355,5015,22.712732,7.078764
C,1183,13850,75.687780,8.541516
D,21,348,1.343570,6.034483
Е,2,22,0.127959,9.090909


## Вывод

Самая большая категория заёмщиков - С. 75.6% от всех просрочек приходятся именно на них, и самый высокий процент просрочек среди других групп - 22.7%. Исходя из таблицы, уровень дохода не слишком сильно влияет на количество просрочек по займам.

## Общий вывод

Мы изучили таблицу с данными по заёмщикам. Обработав данные, мы пришли к следующим выводам:

С ростом количества детей процент задолженности падает, то есть чем больше детей, тем меньше вероятность образования задолженности.
Разведенные и вдовы(-цы) - наиболее склонны возвращать кредиты вовремя, тогда как женатые/замужние люди - самая рискованная группа после тех, кто состоит в гражданском браке или не состоит в браке вовсе.
Уровень дохода не слишком сильно влияет на количество просрочек по займам.

Также мы избавились от дубликатов в таблице, явных и неявных пропусков данных, привели все значения к единому формату, разбили данные по категориям и избавились от крупных ошибок в данных о стаже заёмщиков.